In [ ]:
# imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os

In [ ]:
# settings
sns.set()
%matplotlib inline

In [ ]:
# extract the archive; change directory; show csv files
![[ ! -d CSVs ]] && tar -xvf csvs.tar.xz || echo "Archive already extracted!"

%cd CSVs/
!ls *.csv


In [ ]:
# load VAERSDATA CSV
VAERS_DATA = pd.read_csv('2021VAERSDATA.csv', encoding='cp1252')
VAERS_DATA.head(5)

In [ ]:
# VAERS_DATA CSV Basic Information
print('VAERS_DATA:'); print()
print('Column Headers:\t', VAERS_DATA.columns); print()
print('Rows, Columns:\t', VAERS_DATA.shape); print()
print('Women reporting adverse reactions:\t', len([x for x in VAERS_DATA['SEX'] if x == 'F']))
print('Men reporting adverse reactions:\t', len([x for x in VAERS_DATA['SEX'] if x == 'M']))
print()

DIED_DF = VAERS_DATA[VAERS_DATA['DIED']=='Y']
DEATH_IDS = DIED_DF['VAERS_ID']
print('number of deaths:\t', len(DEATH_IDS))

ER_VISIT_DF = VAERS_DATA[VAERS_DATA['ER_VISIT']=='Y']
ER_VISIT_IDS = ER_VISIT_DF['VAERS_ID']
print('number of er visits:\t', len(ER_VISIT_IDS))

DISABLE_DF = VAERS_DATA[VAERS_DATA['DISABLE']=='Y']
DISABLE_IDS = DISABLE_DF['VAERS_ID']
print('number of disabilities:\t', len(DISABLE_IDS))

In [ ]:
DEATHS = []
for key in DEATH_IDS.keys():
    DEATHS.append(VAERS_DATA.loc[VAERS_DATA['VAERS_ID']==DEATH_IDS[key]])

ages, male_fatalities, female_fatalities = [], [], []
for DEATH in DEATHS:
    if (DEATH['SEX'] == 'M').bool():
        male_fatalities.append(DEATH)
    elif (DEATH['SEX'] == 'F').bool:
        female_fatalities.append(DEATH)
    try:
        ages.append(int(DEATH['AGE_YRS']))
    except ValueError:
        pass

print('Male fatalities:\t', len(male_fatalities))
print('Female fatalities:\t', len(female_fatalities))
print('Average age:\t', np.mean(ages))
print('Standard deviation of age:\t', np.std(ages))


In [ ]:
plt.title('Age of Vaccine Deaths:')
plt.hist(ages)

In [ ]:
er_visits = []
for key in ER_VISIT_IDS.keys():
    er_visits.append(VAERS_DATA.loc[VAERS_DATA['VAERS_ID']==ER_VISIT_IDS[key]])

age_hospitalized, men_hospitalized, women_hospitalized = [], [], []
for visit in er_visits:
    if (visit['SEX'] == 'M').bool():
        men_hospitalized.append(visit)
    elif (visit['SEX'] == 'F').bool():
        women_hospitalized.append(visit)
    try:
        age_hospitalized.append(int(visit['AGE_YRS']))
    except ValueError:
        pass

print('Men hospitalized:\t', len(men_hospitalized))
print('Women hospitalized:\t', len(women_hospitalized))
print('Average age:\t', np.mean(age_hospitalized))
print('Standard deviation of age:\t', np.std(age_hospitalized))

In [ ]:
plt.title('Age of vaccine hospitalizations:')
plt.hist(age_hospitalized)

In [ ]:
disabilities = []
for key in DISABLE_IDS.keys():
    disabilities.append(VAERS_DATA.loc[VAERS_DATA['VAERS_ID']==DISABLE_IDS[key]])

age_disabled, men_disabled, women_disabled = [], [], []
for disability in disabilities:
    if (disability['SEX'] == 'M').bool():
        men_disabled.append(disability)
    elif (disability['SEX'] == 'F').bool():
        women_disabled.append(disability)
    try:
        age_disabled.append(int(disability['AGE_YRS']))
    except ValueError:
        pass

print('Men disabled:\t', len(men_disabled))
print('Women disabled:\t', len(women_disabled))
print('Average age:\t', np.mean(age_disabled))
print('Standard deviation of age:\t', np.std(age_disabled))

In [ ]:
plt.title('Age of vaccine disability:')
plt.hist(age_disabled)

In [ ]:
# load VAERS SYMPTOMS CSV
VAERS_SYMPTOMS = pd.read_csv('2021VAERSSYMPTOMS.csv', encoding='cp1252')
VAERS_SYMPTOMS.head(5)

In [ ]:
#print basic information on VAERS SYMPTOMS CSV
print('VAERS_SYMPTOMS:'); print()
print('Columns:', VAERS_SYMPTOMS.columns); print()
print('Shape:', VAERS_SYMPTOMS.shape); print()
print(len(VAERS_SYMPTOMS['SYMPTOM1'].unique()))
print(len(VAERS_SYMPTOMS['SYMPTOM2'].unique()))
print(len(VAERS_SYMPTOMS['SYMPTOM3'].unique()))
print(len(VAERS_SYMPTOMS['SYMPTOM4'].unique()))
print(len(VAERS_SYMPTOMS['SYMPTOM5'].unique()))

In [ ]:
# load VAERS VAX CSV
VAERS_VAX = pd.read_csv('2021VAERSVAX.csv', encoding='cp1252')
VAERS_VAX.head(5)

In [ ]:
# print basic information on VAERS VAX CSV
print('VAERS_VAX:'); print()
print('Columns:', VAERS_VAX.columns); print()
print('Shape:', VAERS_VAX.shape); print()

In [ ]:
# load COVID DEATHS CSV
COVID_DEATHS = pd.read_csv('Conditions_Contributing_to_COVID-19_Deaths__by_State_and_Age.csv')
COVID_DEATHS.head(5)

In [ ]:
# print basic information on COVID DEATHS CSV
print('Column headers:\t', COVID_DEATHS.columns); print()
print('Rows, Columns:\t', COVID_DEATHS.shape); print()
print('Condition Groups:\t', COVID_DEATHS['Condition Group'].unique()); print()
print('Conditions:\t', COVID_DEATHS['Condition'].unique()); print()
print('Age Groups:\t', COVID_DEATHS['Age Group'].unique()); print()